In [31]:
# import statements

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pickle
from math import comb
import statistics

In [55]:
ibs_graph = pickle.load(open("ibs_graph.pkl", "rb"))
ibd_graph = pickle.load(open("ibd_graph.pkl", "rb"))

In [65]:
# CDF (cumulative distribution function)
# calculate Pr(v <= x): probability a node v has <= x degree
# MARKOVS INEQUALITY - MAKE SURE WE INCLUDE IN PAPER

n = len(ibs_graph.nodes)
m = len(ibd_graph.nodes)
nm_max = max(n, m)
expected = (nm_max) / 2

degree_dict = {}

cdf = np.zeros(nm_max)
cdf_ibd = np.zeros(nm_max)
max_deg_ibs = 0
max_deg_ibd = 0

for node in ibs_graph.nodes:
    degree = len(ibs_graph.edges(node))
    cdf[degree] += 1
    if degree > max_deg_ibs:
        max_deg_ibs = degree

for node in ibd_graph.nodes:
    degree = len(ibd_graph.edges(node))
    cdf_ibd[degree] += 1
    if degree > max_deg_ibd:
        max_deg_ibd += 1

max_deg = max(max_deg_ibs, max_deg_ibd)

cdf_ibs_sum = np.zeros(max_deg + 1)
cdf_ibd_sum = np.zeros(max_deg + 1)
markovs = np.zeros(max_deg + 1)

for i in range(max_deg + 1):
    if i > 0:
        cdf_ibs_sum[i] = cdf[i] + cdf_ibs_sum[i-1]
        cdf_ibd_sum[i] = cdf_ibd[i] + cdf_ibd_sum[i-1]

    else:
        cdf_ibs_sum[i] = cdf[i]
        cdf_ibd_sum[i] = cdf_ibd[i]

    if i > 0:
        markovs[i] = 1 - expected / i
    else: 
        markovs[i] = 0
    markovs[i] = max(0, markovs[i])
    markovs[i] = min(1, markovs[i])

degrees = np.arange(0, max_deg + 1, 1)

cdf_ibs_sum = cdf_ibs_sum / n
cdf_ibd_sum = cdf_ibd_sum / m

# print(degrees)
print(cdf_ibs_sum)
print(cdf_ibd_sum)
# print(cdf_ibd[0])
print(markovs)


[0.8313253  0.86919105 0.88468158 0.90361446 0.91222031 0.91738382
 0.92771084 0.93459552 0.93975904 0.94492255 0.94664372 0.95180723
 0.9535284  0.95524957 0.95869191 0.96213425 0.96901893 0.9707401
 0.9707401  0.97418244 0.98106713 0.98106713 0.9827883  0.9827883
 0.98795181 0.98967298 0.99311532 0.99483649 0.99655766 0.99827883
 0.99827883 1.        ]
[0.80582524 0.82847896 0.84466019 0.85760518 0.87702265 0.88349515
 0.88349515 0.89967638 0.90291262 0.90291262 0.90938511 0.91262136
 0.92556634 0.92880259 0.94174757 0.94498382 0.94498382 0.95145631
 0.95145631 0.95145631 0.95469256 0.95469256 0.95469256 0.96116505
 0.96763754 0.97411003 0.98058252 0.98381877 0.98381877 0.98381877
 0.98381877 0.98381877]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [66]:
# plot

plt.figure(figsize = (15, 15), dpi = 600)
plt.plot(degrees, cdf_ibs_sum, label = "IBS")
plt.plot(degrees, cdf_ibd_sum, label = "IBD")
# plt.plot(degrees, markovs, label = "Markov's")
plt.xlabel("degree")
plt.ylabel("Cumulative Data Function (CDF), Pr(v <= x)")
plt.legend()

In [44]:
# number of edges

m = len(ibs_graph.edges)
print("vertices:", n, "edges:", m, "average degree:", m / n)

deg_list = []
deg_list_nonzero = []

for node in ibs_graph.nodes:
    deg = len(ibs_graph.edges(node))
    deg_list.append(deg)
    if deg > 0:
        deg_list_nonzero.append(deg)

print("median degree:", statistics.median(deg_list), "median nonzero degree:", statistics.median(deg_list_nonzero))
print("clustering coeff:", nx.average_clustering(ibs_graph))

vertices: 581 edges: 417 average degree: 0.7177280550774526
median degree: 0 median nonzero degree: 5.0
clustering coeff: 0.09301932955578972
